In [1]:
from src.dataset import CIFAR10DataModule
from src.models import ModelFactory
from pytorch_lightning.callbacks import ModelPruning
from src.training_module import TrainingModule
from pytorch_lightning.callbacks import Callback
import pytorch_lightning as pl
from src.prune_scheduler import AgpPruningRate
import torchmetrics
from torch import nn
import torch.nn.utils.prune as prune
import torch
from typing import Callable
from pytorch_lightning.loggers.base import LightningLoggerBase
import csv

In [2]:
class SparsityLogger(LightningLoggerBase):
    def __init__(self, file_path, header=['epoch', 'layer', 'sparsity']):
        super().__init__
        self.file_path = file_path

        with open(self.file_path, 'w') as f:
          writer = csv.writer(f)
          writer.writerow(header)
    
    def log(self, metrics):
        fields = [metrics["epoch"], metrics["layer"], metrics["sparsity"]]
        filename = self.file_path
        with open(filename, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
    def experiment(self):
        pass
    def log_hyperparams(self, params, *args, **kwargs):
        pass
    def log_metrics(self, metrics, step):
        pass
    def name(self) -> str:
        pass
    def version(self):
        pass

In [3]:
og_logger = SparsityLogger('og_logger_test.csv')

In [4]:
class PruningTrainingModule(TrainingModule):
    def __init__(
        self, 
        model_name, 
        image_size, 
        num_classes, 
        lr, 
        momentum, 
        epochs,
        weight_decay,
        mixup,
        logger,
        pre_trained=False,
    ):
        super(TrainingModule, self).__init__()
        self.lr = lr
        self.image_size = image_size
        self.num_classes = num_classes
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.epochs = epochs
        self.mixup = mixup
        self._model = self.create_model(model_name=model_name, pre_trained=pre_trained)
        self._loss = nn.CrossEntropyLoss()
        acc = torchmetrics.Accuracy()
        self.val_acc = acc.clone()
        self.train_acc = acc.clone()

        self.freq = 2
        self.prune_end = int(120 * 0.75)
        self.prune_sch = AgpPruningRate(.05, .50, 1, self.prune_end, self.freq)
        self.prune_layers = [module for module in self._model.modules()][:-1]
        self._logger = logger

    def on_train_epoch_start(self) -> None:
        log_dict = {
            'epoch': self.current_epoch,
            'layer': 'start',
            'sparsity': self.freq
        }
        self._logger.log(log_dict)
        if self.current_epoch % self.freq == 1 and self.current_epoch <= self.prune_end:
            target = self.prune_sch.step(self.current_epoch)
            print(target)
            print(f'pruning {target * 100}% sparsity')
            log_dict = {
                'epoch': 'target',
                'layer': '0',
                'sparsity': target
            }
            self._logger.log(log_dict)
            if self.current_epoch > 1 and self.current_epoch < self.prune_end:
                for i, layer in enumerate(self.prune_layers):
                    if type(layer) == nn.Conv2d or type(layer) == nn.Linear:
                        prune.remove(layer, "weight")
            for i, layer in enumerate(self.prune_layers):
                if type(layer) == nn.Conv2d or type(layer) == nn.Linear:
                    prune.l1_unstructured(layer, name='weight', amount=float(target))
                    layer_spar = float(torch.sum(layer.weight == 0))
                    layer_spar /= float(layer.weight.nelement())
                    log_dict = {
                        'epoch': self.current_epoch,
                        'layer': layer,
                        'sparsity': layer_spar
                    }
                    self._logger.log(log_dict)
                    print(f"Sparsity in layer {i} {type(layer)} {layer_spar: 3f}")
        elif self.current_epoch > self.prune_end:
            print("All done pruning")

    def OG_Pruning_Scheduler(self) -> Callable:
        if self.current_epoch % self.freq == 1 and self.current_epoch <= self.prune_end:
            target = self.prune_sch.step(self.current_epoch)
            return target



In [ ]:
class OG_Pruning_Callback(Callback):    
    def on_train_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        if trainer.current_epoch % pl_module.freq == 1 and trainer.current_epoch <= pl_module.prune_end:
            target = pl_module.prune_sch.step(trainer.current_epoch)
            print(target)
            print(f'pruning {target * 100}% sparsity')
            if trainer.current_epoch > 1 and trainer.current_epoch < pl_module.prune_end:
                for i, layer in enumerate(pl_module.prune_layers):
                    if type(layer) == nn.Conv2d or type(layer) == nn.Linear:
                        prune.remove(layer, "weight")
            for i, layer in enumerate(pl_module.prune_layers):
                if type(layer) == nn.Conv2d or type(layer) == nn.Linear:
                    prune.l1_unstructured(layer, name='weight', amount=float(target))
                    layer_spar = float(torch.sum(layer.weight == 0))
                    layer_spar /= float(layer.weight.nelement())
                    print(f"Sparsity in layer {i} {type(layer)} {layer_spar: 3f}")
        elif trainer.current_epoch > pl_module.prune_end:
            print("All done pruning")
    
    def on_train_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        torch.save(trainer.model.state_dict(), 'og_pruning_weights.pth')

In [5]:
training_module = PruningTrainingModule(
    model_name='resnet34',
    image_size=1,
    num_classes=10,
    pre_trained=False,
    lr=0.01,
    epochs=20,
    mixup=False,
    momentum=0.005,
    logger=og_logger,
    weight_decay=1e-5
        )

In [6]:
dm = CIFAR10DataModule(data_dir='data/', num_workers=128, pin_memory=True)

In [7]:
og_trainer = pl.Trainer(gpus=1, max_epochs=20)

og_trainer.fit(training_module, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | _model    | ResNet           | 21.3 M
1 | _loss     | CrossEntropyLoss | 0     
2 | val_acc   | Accuracy         | 0     
3 | train_acc | Accuracy         | 0     
-----------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.128    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/nathaniel/anaconda3/envs/kd-experiments-pl/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 128 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

0.25353117956423743
pruning 25.353117956423745% sparsity
Sparsity in layer 1 <class 'torch.nn.modules.conv.Conv2d'>  0.253472
Sparsity in layer 7 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 10 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 13 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 16 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 19 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 22 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 26 <class 'torch.nn.modules.conv.Conv2d'>  0.253526
Sparsity in layer 29 <class 'torch.nn.modules.conv.Conv2d'>  0.253533
Sparsity in layer 32 <class 'torch.nn.modules.conv.Conv2d'>  0.253540
Sparsity in layer 35 <class 'torch.nn.modules.conv.Conv2d'>  0.253533
Sparsity in layer 38 <class 'torch.nn.modules.conv.Conv2d'>  0.253533
Sparsity in layer 41 <class 'torch.nn.modules.conv.Conv2d'>  0.253533
Sparsity in layer 44 <class 'torch.

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

0.27350500093914354
pruning 27.350500093914356% sparsity
Sparsity in layer 1 <class 'torch.nn.modules.conv.Conv2d'>  0.273727
Sparsity in layer 7 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 10 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 13 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 16 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 19 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 22 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 26 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 29 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 32 <class 'torch.nn.modules.conv.Conv2d'>  0.273560
Sparsity in layer 35 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 38 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 41 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 44 <class 'torch.

Validating: 0it [00:00, ?it/s]

In [8]:
pl_logger = SparsityLogger('pl_logger_test.csv')

In [9]:
class PL_Pruning_Callback(Callback):
   def on_train_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule") -> None:
        torch.save(trainer.model.state_dict(), 'pl_pruning_weights.pth')

pl_training_module = PruningTrainingModule(
    model_name='resnet34',
    image_size=1,
    num_classes=10,
    pre_trained=False,
    lr=0.01,
    epochs=20,
    mixup=False,
    momentum=0.005,
    logger=pl_logger,
    weight_decay=1e-5
        )

pl_pruning_trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[PL_Pruning_Callback(), ModelPruning(pruning_fn="l1_unstructured", parameter_names=["weight"], prune_on_train_epoch_end=False, make_pruning_permanent=True, amount=pl_training_module.OG_Pruning_Scheduler(), apply_pruning=True, verbose=2)])

pl_pruning_trainer.fit(pl_training_module ,dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/nathaniel/anaconda3/envs/kd-experiments-pl/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | _model    | ResNet           | 21.3 M
1 | _loss     | CrossEntropyLoss | 0     
2 | val_acc   | Accuracy         | 0     
3 | train_acc | Accuracy         | 0     
-----------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.128    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

0.04999999999999999
pruning 4.999999999999999% sparsity
Sparsity in layer 1 <class 'torch.nn.modules.conv.Conv2d'>  0.049769
Sparsity in layer 7 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 10 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 13 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 16 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 19 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 22 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 26 <class 'torch.nn.modules.conv.Conv2d'>  0.049995
Sparsity in layer 29 <class 'torch.nn.modules.conv.Conv2d'>  0.050001
Sparsity in layer 32 <class 'torch.nn.modules.conv.Conv2d'>  0.050049
Sparsity in layer 35 <class 'torch.nn.modules.conv.Conv2d'>  0.050001
Sparsity in layer 38 <class 'torch.nn.modules.conv.Conv2d'>  0.050001
Sparsity in layer 41 <class 'torch.nn.modules.conv.Conv2d'>  0.050001
Sparsity in layer 44 <class 'torch.n

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

0.27350500093914354
pruning 27.350500093914356% sparsity
Sparsity in layer 1 <class 'torch.nn.modules.conv.Conv2d'>  0.273727
Sparsity in layer 7 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 10 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 13 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 16 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 19 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 22 <class 'torch.nn.modules.conv.Conv2d'>  0.273492
Sparsity in layer 26 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 29 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 32 <class 'torch.nn.modules.conv.Conv2d'>  0.273560
Sparsity in layer 35 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 38 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 41 <class 'torch.nn.modules.conv.Conv2d'>  0.273505
Sparsity in layer 44 <class 'torch.

Validating: 0it [00:00, ?it/s]

/home/nathaniel/anaconda3/envs/kd-experiments-pl/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
